In this notebook we continue data cleaning. We drop any columns that contained empty data which would be an error from the data scraping process. We convert the duration column into a numerical column that lists the duration of the short in seconds. We add a few post performance metrics such as a column that lists likes/subscriber. Finally, we make sure that our data set only contains shorts from 3/1/2024 to 3/1/2025 which is the time period of posts that we wanted to look at for our project. 

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#importing our slightly cleaned data
#Changed to a relative directory so anyone can run it
df = pd.read_csv("../data/clean_data2.csv")
print(df.columns) #Investigating which columns remain after cleaning 

Index(['channelDescription', 'channelJoinedDate', 'channelLocation',
       'channelTotalVideos', 'channelTotalViews', 'channelUsername',
       'commentsCount', 'date', 'duration', 'id', 'isChannelVerified', 'likes',
       'numberOfSubscribers', 'order', 'text', 'title', 'url', 'viewCount'],
      dtype='object')


In [34]:
df.info()

df["text"].sample(10)
#THIS IS THE "DESCRIPTION" OF THE VIDEO. MANY USERS LEAVE THE DESCRIPTION BLANK!!! 
#We want this. 
#I am changing the NaN entries to empty strings. 
# This is an easier and more performant way of doing this
# Also doing this for channelDescription
df["text"] = df["text"].fillna("")
df["channelDescription"] = df["channelDescription"].fillna("")

df["text"].sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   channelDescription   18151 non-null  object 
 1   channelJoinedDate    19185 non-null  object 
 2   channelLocation      13922 non-null  object 
 3   channelTotalVideos   19185 non-null  float64
 4   channelTotalViews    19185 non-null  float64
 5   channelUsername      19185 non-null  object 
 6   commentsCount        19163 non-null  float64
 7   date                 19184 non-null  object 
 8   duration             18818 non-null  object 
 9   id                   19185 non-null  object 
 10  isChannelVerified    19185 non-null  bool   
 11  likes                19167 non-null  float64
 12  numberOfSubscribers  19185 non-null  float64
 13  order                19185 non-null  float64
 14  text                 5657 non-null   object 
 15  title                19185 non-null 

5594                                                      
1366     Which perfume do you like the most? 🥰 ‪@Makeup...
8614     Get this Acne Friendly GLOW in just 4 steps us...
16322                                                     
9634     #skincarehaul #theordinary #theordinaryhaul #t...
10714                                                     
4470                                                      
13109                                                     
13370                                                     
18695                                                     
Name: text, dtype: object

In [35]:
#Double-checking for non-null entries
#Let's drop Channel Location since that one is also often empty

# There's no real point in incrementing the name of the dataframe here

df = df.drop("channelLocation", axis = 1) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19185 entries, 0 to 19184
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   channelDescription   19185 non-null  object 
 1   channelJoinedDate    19185 non-null  object 
 2   channelTotalVideos   19185 non-null  float64
 3   channelTotalViews    19185 non-null  float64
 4   channelUsername      19185 non-null  object 
 5   commentsCount        19163 non-null  float64
 6   date                 19184 non-null  object 
 7   duration             18818 non-null  object 
 8   id                   19185 non-null  object 
 9   isChannelVerified    19185 non-null  bool   
 10  likes                19167 non-null  float64
 11  numberOfSubscribers  19185 non-null  float64
 12  order                19185 non-null  float64
 13  text                 19185 non-null  object 
 14  title                19185 non-null  object 
 15  url                  19185 non-null 

In [36]:
#Okay, now I see that some columns still have empty entries. Let's drop them. 

df = df.dropna().copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18798 entries, 0 to 19184
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   channelDescription   18798 non-null  object 
 1   channelJoinedDate    18798 non-null  object 
 2   channelTotalVideos   18798 non-null  float64
 3   channelTotalViews    18798 non-null  float64
 4   channelUsername      18798 non-null  object 
 5   commentsCount        18798 non-null  float64
 6   date                 18798 non-null  object 
 7   duration             18798 non-null  object 
 8   id                   18798 non-null  object 
 9   isChannelVerified    18798 non-null  bool   
 10  likes                18798 non-null  float64
 11  numberOfSubscribers  18798 non-null  float64
 12  order                18798 non-null  float64
 13  text                 18798 non-null  object 
 14  title                18798 non-null  object 
 15  url                  18798 non-null  obje

In [37]:
#We are interested in engagement metrics. Obviously the more subscribers you have, the more views/likes/comments you will get. A post is performing well
#regardless of follower count if it has good "engagement", which is a metrics such as likes/subscriber. Here I am transforming raw likes/comments/views
#to engagement metrics. 

# Make sure to use underscores in column names. Spaces interfere with some functions such as pd.query()

df["likes_per_subscriber"] = df["likes"]/df["numberOfSubscribers"]
df["comments_per_subscriber"] = df["commentsCount"]/df["numberOfSubscribers"]
df["views_per_subscriber"] = df["viewCount"]/df["numberOfSubscribers"]


#Changing duration column from a string to an integer 
import datetime 

def convert_duration_to_int(item):
    item_as_datetime = datetime.datetime.strptime( item , "%H:%M:%S"  ) #converts string to a datetime object
    seconds = item_as_datetime.second + 60 * item_as_datetime.minute + 3600 * item_as_datetime.hour
    return seconds
    
df["duration_in_seconds"] = df["duration"].apply(convert_duration_to_int) 
df[["duration", "duration_in_seconds"]].sample(10) #Can confirm that column is correct now

,duration,duration_in_seconds
1226,00:00:55,55
6011,00:00:13,13
14447,00:00:38,38
6665,00:00:28,28
13410,00:00:14,14
14211,00:00:07,7
29,00:01:00,60
14269,00:00:16,16
17903,00:01:00,60
2704,00:01:00,60


In [38]:
#We should change our data so that none of the data comes after 3/1/2025 and none comes before 3/1/2024. 

In [ ]:
#Okay, let's restrict our attention to those rows that came after October 15th.
#At the same time, we should change our data so that none of the data comes after 3/1. This is the date we first started collecting
#and after that date, data was scraped over the first few days. 

#Okay, time to continue to clean by removing data after 3/1/2025 and before 3/1/2024.
#First, we need to change the date column so that it is a datetime object and not a string

df["datetime"] = df["date"].apply(datetime.datetime.fromisoformat)
df["datetime_date"] = df["datetime"].dt.date.astype(str)
df["datetime_date"]

df = df.query("datetime_date < '2025-03-01'")
df = df.query("datetime_date > '2024-03-01'")

In [ ]:
df.to_csv("../data/cleaned_data_correct_dates.csv")